In [1]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

### Set the data for the tests

In [2]:
# read Ospiti.csv
df = pd.read_csv('input/Ospiti.csv')
df["Data"] = pd.to_datetime(df["Data"]).dt.date
df.set_index("Data",inplace=True)
df = df.iloc[:-1]
df

,Ospiti
Data,
2021-06-01,319.0
2021-06-02,396.0
2021-06-03,188.0
2021-06-04,258.0
2021-06-05,571.0
...,...
2022-11-25,375.0
2022-11-26,732.0
2022-11-27,495.0


In [3]:
# divide the dataframe incrementally by one day (start with 365 days)
df_list = []
for i in range(0, len(df) - 364, 1):
    df_list.append(df.iloc[:i+365])
df_list

[            Ospiti
 Data              
 2021-06-01   319.0
 2021-06-02   396.0
 2021-06-03   188.0
 2021-06-04   258.0
 2021-06-05   571.0
 ...            ...
 2022-05-28   646.0
 2022-05-29   579.0
 2022-05-30   255.0
 2022-05-31   329.0
 2022-06-01   339.0
 
 [365 rows x 1 columns],
             Ospiti
 Data              
 2021-06-01   319.0
 2021-06-02   396.0
 2021-06-03   188.0
 2021-06-04   258.0
 2021-06-05   571.0
 ...            ...
 2022-05-29   579.0
 2022-05-30   255.0
 2022-05-31   329.0
 2022-06-01   339.0
 2022-06-02   557.0
 
 [366 rows x 1 columns],
             Ospiti
 Data              
 2021-06-01   319.0
 2021-06-02   396.0
 2021-06-03   188.0
 2021-06-04   258.0
 2021-06-05   571.0
 ...            ...
 2022-05-30   255.0
 2022-05-31   329.0
 2022-06-01   339.0
 2022-06-02   557.0
 2022-06-03   357.0
 
 [367 rows x 1 columns],
             Ospiti
 Data              
 2021-06-01   319.0
 2021-06-02   396.0
 2021-06-03   188.0
 2021-06-04   258.0
 2021-06-05   571.0

In [4]:
len(df_list)

182

### Get average predictions auto arima

In [4]:
import my_auto_arima

# create a dataframe with the predictions for each day
df_pred_autoarima = pd.DataFrame()
for i in range(len(df_list)):
    df_pred_autoarima = df_pred_autoarima.append(my_auto_arima.predictions(df_list[i].reset_index()));
    #df_pred_autoarima = pd.concat([df_pred_autoarima, my_auto_arima.predictions(df_list[i].reset_index())], axis=1);

/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence problem: minimize gave code 2]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/statsforecast/arima.py:861: UserWarning: possible convergence probl

In [7]:
df_pred_autoarima = df_pred_autoarima.groupby(by=df_pred_autoarima.index).mean()
df_pred_autoarima

,AutoArima Forecasts
Data,
2022-06-02,397.950219
2022-06-03,459.580753
2022-06-04,634.782739
2022-06-05,523.540613
2022-06-06,217.089558
...,...
2022-12-02,409.874366
2022-12-03,760.022672
2022-12-04,485.478291


In [10]:
# save the predictions in a csv file
df_pred_autoarima.to_csv('output/predictions_autoarima_avg.csv')

### Get average predictions Dense NN

In [5]:
import my_dense_model

# create a dataframe with the predictions for each day
df_pred_dense = pd.DataFrame()
for i in range(len(df_list)):
    df_pred_dense = df_pred_dense.append(my_dense_model.predictions(df_list[i].reset_index()));
    #df_pred_autoarima = pd.concat([df_pred_autoarima, my_auto_arima.predictions(df_list[i].reset_index())], axis=1);

2022-12-16 18:08:43.974170: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-16 18:08:43.974324: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (davide-Inspiron-3891): /proc/driver/nvidia/version does not exist
2022-12-16 18:08:43.975573: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
df_pred_dense = df_pred_dense.groupby(by=df_pred_dense.index).mean()
df_pred_dense

,Dense NN forecasts
2022-06-02,372.704895
2022-06-03,414.354248
2022-06-04,520.879028
2022-06-05,499.169250
2022-06-06,355.031464
...,...
2022-12-02,409.899475
2022-12-03,603.825745
2022-12-04,440.580933
2022-12-05,260.579010


In [7]:
# save the predictions in a csv file
df_pred_dense.to_csv('output/predictions_dense_avg.csv')

### Get average predictions Dense NN + exogenous

In [2]:
# read Ospiti.csv
df = pd.read_csv('input/Ospiti.csv')
df["Data"] = pd.to_datetime(df["Data"]).dt.date
df.set_index("Data",inplace=True)
df = df.iloc[:-1]
df

,Ospiti
Data,
2021-06-01,319.0
2021-06-02,396.0
2021-06-03,188.0
2021-06-04,258.0
2021-06-05,571.0
...,...
2022-11-25,375.0
2022-11-26,732.0
2022-11-27,495.0


In [3]:
ex = pd.read_csv('input/Exogenous.csv', index_col=0)
ex.rename(columns={"date":"Data"},inplace=True)
ex["Data"] = pd.to_datetime(ex["Data"]).dt.date
ex.set_index("Data",inplace=True)
ex = ex.iloc[:-2]
ex

,Vacanze,Viaggi,Gita,Google Maps,Traffico,Traffico Milano,Mare,Montagna,Lago Garda,Santo Ambrogio,...,champions league,Partita calcio,Previsioni del tempo,Pioggia,Neve,Meteo,Meteo Agrate,autostrada A4,Pizza,Ristorante
Data,,,,,,,,,,,,,,,,,,,,,
2021-06-01,6.125000,3.708333,3.500000,4.458333,2.000000,1.791667,16.208333,2.333333,0.000000,1.666667,...,3.250000e+00,0.000000,1.250000,2.916667,1.750000,17.333333,2.458333,6.500000,0.000000,13.458333
2021-06-02,7.458333,5.660714,5.309524,3.386905,5.000000,1.148810,17.755952,3.976190,0.000000,1.142857,...,2.107143e+00,0.000000,4.583333,2.511905,1.107143,19.904762,1.220238,7.690476,0.000000,16.458333
2021-06-03,7.296171,7.753378,3.229730,3.922297,3.576577,1.246622,18.762387,3.518018,0.000000,5.099099,...,2.132883e+00,0.000000,3.934685,1.894144,1.619369,19.063063,3.575450,5.644144,0.000000,14.800676
2021-06-04,6.883095,6.715952,3.252381,4.179286,3.975238,1.110714,19.611190,4.105714,0.000000,4.560000,...,3.166190e+00,0.289524,4.034762,4.767143,1.971905,22.666667,2.877381,4.875238,0.000000,14.538333
2021-06-05,6.716549,5.798389,2.486769,4.803404,2.972685,1.006135,18.208974,3.593043,0.000000,4.786172,...,3.767286e+00,0.344857,3.886790,3.491357,2.513764,27.323944,5.621799,3.878148,0.000000,16.164693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-25,4.367012,2.657099,2.094199,11.293774,1.631865,2.807394,4.216981,0.028546,4.148074,0.021223,...,4.444330e-21,3.063728,0.474055,2.775318,5.455785,16.288040,3.553792,1.075770,3.299307,17.058101
2022-11-26,3.275259,2.701158,1.570649,11.803664,1.223898,2.855545,4.016902,0.021409,4.111055,0.015917,...,3.333248e-21,3.568629,0.355541,2.956489,5.414756,15.070197,3.029928,0.806827,2.474480,20.189409
2022-11-27,2.456444,2.702952,1.177987,10.633998,0.917924,2.933326,3.543927,0.016057,4.135375,0.011938,...,2.499936e-21,3.270222,0.266656,2.779866,5.863150,14.313064,2.907862,0.605121,1.855860,20.735807


In [4]:
df_tot = pd.concat([df, ex], axis=1)
df_tot

,Ospiti,Vacanze,Viaggi,Gita,Google Maps,Traffico,Traffico Milano,Mare,Montagna,Lago Garda,...,champions league,Partita calcio,Previsioni del tempo,Pioggia,Neve,Meteo,Meteo Agrate,autostrada A4,Pizza,Ristorante
Data,,,,,,,,,,,,,,,,,,,,,
2021-06-01,319.0,6.125000,3.708333,3.500000,4.458333,2.000000,1.791667,16.208333,2.333333,0.000000,...,3.250000e+00,0.000000,1.250000,2.916667,1.750000,17.333333,2.458333,6.500000,0.000000,13.458333
2021-06-02,396.0,7.458333,5.660714,5.309524,3.386905,5.000000,1.148810,17.755952,3.976190,0.000000,...,2.107143e+00,0.000000,4.583333,2.511905,1.107143,19.904762,1.220238,7.690476,0.000000,16.458333
2021-06-03,188.0,7.296171,7.753378,3.229730,3.922297,3.576577,1.246622,18.762387,3.518018,0.000000,...,2.132883e+00,0.000000,3.934685,1.894144,1.619369,19.063063,3.575450,5.644144,0.000000,14.800676
2021-06-04,258.0,6.883095,6.715952,3.252381,4.179286,3.975238,1.110714,19.611190,4.105714,0.000000,...,3.166190e+00,0.289524,4.034762,4.767143,1.971905,22.666667,2.877381,4.875238,0.000000,14.538333
2021-06-05,571.0,6.716549,5.798389,2.486769,4.803404,2.972685,1.006135,18.208974,3.593043,0.000000,...,3.767286e+00,0.344857,3.886790,3.491357,2.513764,27.323944,5.621799,3.878148,0.000000,16.164693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-26,732.0,3.275259,2.701158,1.570649,11.803664,1.223898,2.855545,4.016902,0.021409,4.111055,...,3.333248e-21,3.568629,0.355541,2.956489,5.414756,15.070197,3.029928,0.806827,2.474480,20.189409
2022-11-27,495.0,2.456444,2.702952,1.177987,10.633998,0.917924,2.933326,3.543927,0.016057,4.135375,...,2.499936e-21,3.270222,0.266656,2.779866,5.863150,14.313064,2.907862,0.605121,1.855860,20.735807
2022-11-28,226.0,1.842333,2.568880,0.883490,10.121332,0.688443,3.033328,4.324612,0.012043,3.768198,...,1.874952e-21,4.567250,0.199992,2.282817,5.532779,14.495215,4.472563,0.453840,1.391895,19.697688


In [5]:
# remove rows with NaN values
df_tot = df_tot.dropna()
df_tot

,Ospiti,Vacanze,Viaggi,Gita,Google Maps,Traffico,Traffico Milano,Mare,Montagna,Lago Garda,...,champions league,Partita calcio,Previsioni del tempo,Pioggia,Neve,Meteo,Meteo Agrate,autostrada A4,Pizza,Ristorante
Data,,,,,,,,,,,,,,,,,,,,,
2021-06-01,319.0,6.125000,3.708333,3.500000,4.458333,2.000000,1.791667,16.208333,2.333333,0.000000,...,3.250000e+00,0.000000,1.250000,2.916667,1.750000,17.333333,2.458333,6.500000,0.000000,13.458333
2021-06-02,396.0,7.458333,5.660714,5.309524,3.386905,5.000000,1.148810,17.755952,3.976190,0.000000,...,2.107143e+00,0.000000,4.583333,2.511905,1.107143,19.904762,1.220238,7.690476,0.000000,16.458333
2021-06-03,188.0,7.296171,7.753378,3.229730,3.922297,3.576577,1.246622,18.762387,3.518018,0.000000,...,2.132883e+00,0.000000,3.934685,1.894144,1.619369,19.063063,3.575450,5.644144,0.000000,14.800676
2021-06-04,258.0,6.883095,6.715952,3.252381,4.179286,3.975238,1.110714,19.611190,4.105714,0.000000,...,3.166190e+00,0.289524,4.034762,4.767143,1.971905,22.666667,2.877381,4.875238,0.000000,14.538333
2021-06-05,571.0,6.716549,5.798389,2.486769,4.803404,2.972685,1.006135,18.208974,3.593043,0.000000,...,3.767286e+00,0.344857,3.886790,3.491357,2.513764,27.323944,5.621799,3.878148,0.000000,16.164693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-25,375.0,4.367012,2.657099,2.094199,11.293774,1.631865,2.807394,4.216981,0.028546,4.148074,...,4.444330e-21,3.063728,0.474055,2.775318,5.455785,16.288040,3.553792,1.075770,3.299307,17.058101
2022-11-26,732.0,3.275259,2.701158,1.570649,11.803664,1.223898,2.855545,4.016902,0.021409,4.111055,...,3.333248e-21,3.568629,0.355541,2.956489,5.414756,15.070197,3.029928,0.806827,2.474480,20.189409
2022-11-27,495.0,2.456444,2.702952,1.177987,10.633998,0.917924,2.933326,3.543927,0.016057,4.135375,...,2.499936e-21,3.270222,0.266656,2.779866,5.863150,14.313064,2.907862,0.605121,1.855860,20.735807


In [6]:
# divide the dataframe incrementally by one day (start with 365 days)
df_list = []
for i in range(0, len(df_tot) - 364, 1):
    df_list.append(df_tot.iloc[:i+365])
df_list

[            Ospiti   Vacanze    Viaggi      Gita  Google Maps  Traffico  \
 Data                                                                      
 2021-06-01   319.0  6.125000  3.708333  3.500000     4.458333  2.000000   
 2021-06-02   396.0  7.458333  5.660714  5.309524     3.386905  5.000000   
 2021-06-03   188.0  7.296171  7.753378  3.229730     3.922297  3.576577   
 2021-06-04   258.0  6.883095  6.715952  3.252381     4.179286  3.975238   
 2021-06-05   571.0  6.716549  5.798389  2.486769     4.803404  2.972685   
 ...            ...       ...       ...       ...          ...       ...   
 2022-05-28   646.0  1.282297  0.008534  3.468020     4.828658  2.328381   
 2022-05-29   579.0  0.961723  0.006401  3.548932     5.631910  2.215036   
 2022-05-30   255.0  0.721292  0.004800  3.932532     7.557266  1.921693   
 2022-05-31   329.0  1.863886  0.003600  2.949399     8.027949  1.791270   
 2022-06-01   339.0  3.512498  0.002700  2.212049     8.281379  3.249702   
 
          

In [7]:
len(df_list)

182

In [9]:
import my_dense_model_ex

# create a dataframe with the predictions for each day
df_pred_dense_ex = pd.DataFrame()
for i in range(len(df_list)):
    df_pred_dense_ex = df_pred_dense_ex.append(my_dense_model_ex.predictions(df_list[i]));
    #df_pred_autoarima = pd.concat([df_pred_autoarima, my_auto_arima.predictions(df_list[i].reset_index())], axis=1);

In [10]:
df_pred_dense_ex = df_pred_dense_ex.groupby(by=df_pred_dense_ex.index).mean()
df_pred_dense_ex

,Dense NN forecasts
2022-06-02,300.758087
2022-06-03,330.540405
2022-06-04,567.003540
2022-06-05,554.039856
2022-06-06,324.335632
...,...
2022-12-02,360.610504
2022-12-03,715.683228
2022-12-04,582.777954
2022-12-05,329.764557


In [11]:
# save the predictions in a csv file
df_pred_dense_ex.to_csv('output/predictions_dense_ex_avg.csv')